## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [17]:
from google.colab import files
uploaded = files.upload()


KeyboardInterrupt: 

In [5]:
from dotenv import load_dotenv
import os

load_dotenv(".env")
groq_key = os.getenv("groq_api_key")
print("Key loaded:", groq_key[:5] + "..." if groq_key else "Key not found")
with open(".env", "r") as f:
    print(f.read())


Key loaded: gsk_Q...
groq_api_key=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


In [12]:

from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a72d08ec9d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a72d08d77d0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [13]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is kanish chadha and I study at thapar university")])

AIMessage(content="Hi Kanish!\n\nIt's nice to meet you. Thapar University is a well-regarded institution. What are you studying there?  \n\nI'm eager to learn more about you and your interests. \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 25, 'total_tokens': 75, 'completion_time': 0.090909091, 'prompt_time': 0.002149886, 'queue_time': 0.101743487, 'total_time': 0.093058977}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-de4db6f7-c43e-4c46-a4fb-8bf9b1e0c2f7-0', usage_metadata={'input_tokens': 25, 'output_tokens': 50, 'total_tokens': 75})

In [14]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is kanish chadha and I study at thapar university"),
        AIMessage(content="Hi Kanish!\n\nIt's nice to meet you. Thapar University is a well-regarded institution. What are you studying there?  \n\nI'm eager to learn more about you and your interests. \n\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)
## what i provdd here was some context in human messase and aimessage gave me a reposse then i asked him whats my
## name and he answered so we are goona work more on the history part of this

AIMessage(content="You said your name is Kanish Chadha and you study at Thapar University. \n\nIs there anything else you'd like to tell me about yourself? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 94, 'total_tokens': 132, 'completion_time': 0.069090909, 'prompt_time': 0.004318702, 'queue_time': 0.019927206, 'total_time': 0.073409611}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c21a72b4-bcca-47c3-a05a-17012d440da8-0', usage_metadata={'input_tokens': 94, 'output_tokens': 38, 'total_tokens': 132})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [16]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

## so what are doing here is creating a function for checking each sesssion in our chats with our llm models
## so it takes input session_id as string in it and return our whole chat the return here is class "BaseChatMessageHistory"
## if session is not present we add it in store.

In [18]:
## showing one hard encoded message just for the sake of an example
config={"configurable":{"session_id":"chat1"}}

In [19]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Kanish and I am a student at thapar")],
    config=config
)

In [20]:
response.content

"Hi Kanish,\n\nIt's nice to meet you! Thapar University is a great institution. What are you studying there? \n\nIs there anything I can help you with today?  \n\n"

In [21]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Kanish.  😊 \n\nI remember that from our introduction! \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 81, 'total_tokens': 113, 'completion_time': 0.058181818, 'prompt_time': 0.004147712, 'queue_time': 0.020869165000000002, 'total_time': 0.06232953}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c8debc75-b304-4362-9a69-f332ab897c3b-0', usage_metadata={'input_tokens': 81, 'output_tokens': 32, 'total_tokens': 113})

In [23]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

## see here now we went to chat2 and the llm has no converstaiona about us so it shows this

"Since I don't have access to past conversations, I don't know your name.  Would you like to tell me? 😊  \n"

In [25]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content
## now we gave it somme content

"Hi John!  \n\nIs there anything you'd like to ask or talk about? I'm here to help! 😊 \n"

In [26]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  I remember you telling me. 😊  \n\nHow can I help you, John? \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [30]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
        ## before we were giving in a list of messages like human or aimessage but nwo we are giving
        ## but now we are maing a message placeholder where the meessage given from the user is directly into messages.
    ]
)

chain=prompt|model

In [43]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Siddarth")]})

## see this example it is best now we are giving under messages

AIMessage(content="Hello Siddarth! \n\nIt's nice to meet you.  \n\nI'm happy to help with any questions you have. Just ask! 😊  What can I do for you today?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 32, 'total_tokens': 78, 'completion_time': 0.083636364, 'prompt_time': 0.002395755, 'queue_time': 0.019603763, 'total_time': 0.086032119}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b51e69b9-89dd-4b11-a60e-4a1eb228ad7a-0', usage_metadata={'input_tokens': 32, 'output_tokens': 46, 'total_tokens': 78})

In [32]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [33]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Kanish and i like to play basketball")],
    config=config
)

response

AIMessage(content="Hi Kanish, it's nice to meet you!  \n\nI'm glad to hear you like to play basketball.  🏀 \n\nDo you have a favorite team or player?  What position do you like to play?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 38, 'total_tokens': 91, 'completion_time': 0.096363636, 'prompt_time': 0.002337485, 'queue_time': 0.016053065, 'total_time': 0.098701121}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f4d71b2c-b1fb-4736-a33d-070f486fb7ec-0', usage_metadata={'input_tokens': 38, 'output_tokens': 53, 'total_tokens': 91})

In [42]:
config3={"configurable": {"session_id":"chat4"}}
response=with_message_history.invoke(
    [
        HumanMessage(content="Hi its me.")
    ],
    config=config3
)
response


AIMessage(content="Hi! 👋  \n\nIt's good to hear from you. What can I do for you today?  😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 110, 'total_tokens': 138, 'completion_time': 0.050909091, 'prompt_time': 0.005067169, 'queue_time': 0.019967458, 'total_time': 0.05597626}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-07db089e-0607-4b31-a619-cb1e2affffca-0', usage_metadata={'input_tokens': 110, 'output_tokens': 28, 'total_tokens': 138})

In [44]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model
## need to give two inouts now messages and the language u want it to ely u with

In [46]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Kanish")],"language":"Hinglish"})
response.content

"Namaste Kanish! 👋  \n\nIt's nice to meet you!  How can I help you today? 😊  \n\n"

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [47]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [65]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am surya")],"language":"spanish"},
    config=config
)
repsonse.content

'¡Hola Surya! 👋  ¿Qué tal estás?  ¿En qué puedo ayudarte hoy? 😄 \n'

In [62]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "spanish"},
    config=config,
)

In [63]:
response.content

'¡Tu nombre es Surya!  😊  \n\n¿Necesitas algo más?  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [66]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [67]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model

)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal information, including your ice cream preferences!  \n\nWhat's your favorite flavor? 😊\n"

In [68]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2".  \n\nDo you want to try another one? 😄  \n\n'

In [69]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [70]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name. \n\nWould you like to tell me your name? 😊  \n\n"

In [71]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  \n\n"

In [73]:
# 1. Make the folder
!mkdir project_folder

# 2. Move the requirements.txt into the folder
!mv requirements.txt project_folder/

# 3. Move your notebook into the folder
!mv 1-chatbots.ipynb project_folder/



mkdir: cannot create directory ‘project_folder’: File exists
mv: cannot stat 'requirements.txt': No such file or directory
mv: cannot stat '1-chatbots.ipynb': No such file or directory
